To obtain a Hugging Face token, users must create an account on the HuggingFace website, navigate to their profile settings, and generate a new token. Once the token is acquired, users can input it in their code to seamlessly integrate Hugging Face models and functionalities into their projects.

In [2]:
huggingface_token = ""

### **Installing import libraries**

In [3]:
!pip install transformers
!pip install torch
!pip install scikit-learn
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

Found existing installation: unsloth 2024.11.10
Uninstalling unsloth-2024.11.10:
  Successfully uninstalled unsloth-2024.11.10
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-z29ehcyp
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-z29ehcyp
  Resolved https://github.com/unslothai/unsloth.git to commit 8558bc92b06f9128499484ef737fa71b966ffc23
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.11.10-py3-none-any.whl size=166794 sha256=d6c9909e30db96c594fbd0bd27c5dfc396370a6fc95050be4614cda35ed7e989
  Stored in directory: /tmp/pip-ephem-wheel-cache-fzrlbo3s/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


## **Importing Libraries**

In [4]:
import torch
import gc
import pandas as pd
import os
import logging
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorWithPadding, DataCollatorForSeq2Seq
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer
from torch.utils.data import Dataset
from huggingface_hub import login
from google.colab import drive

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## **Creating the DataFrames**

In this block we import all of the datasets that will be used to train and evaluate the LLM. Since the file sizes of the csvs are too large to upload to Colab directly, it is presently configured to read a file called 'clean data' that would be stored on one's Google Drive.

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Mounting Google Drive
drive.mount('/content/drive')

# Defining file path base
base_path = '/content/drive/My Drive/clean data/'

# Loading dataframes from CSV files in Google Drive
dfs = {
    'training_crime': pd.read_csv(base_path + 'training_crime.csv')[['text', 'is_true']],
    'training_health': pd.read_csv(base_path + 'training_health.csv')[['text', 'is_true']],
    'training_politics': pd.read_csv(base_path + 'training_politics.csv')[['text', 'is_true']],
    'training_science': pd.read_csv(base_path + 'training_science.csv')[['text', 'is_true']],
    'training_social': pd.read_csv(base_path + 'training_social.csv')[['text', 'is_true']],
}

# Combining all training dataframes to make one merged training dataset
df_training = pd.concat([dfs['training_crime'], dfs['training_health'], dfs['training_politics'], dfs['training_science'],
                         dfs['training_social']], ignore_index=True)

# Splitting into 70% training and 30% testing
train_df, test_df = train_test_split(df_training, test_size=0.3, random_state=42, stratify=None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Initializing the Model & Tokenizer**

In [6]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    #max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    token = huggingface_token
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = False, # Change to 'unsloth' if you're running out of memory
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
)

==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.11.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## **Defining a Custom Dataset**

In order to handle the text-to-token conversion, we will define a pytorch Dataset.

In [7]:
class NewsDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Get the text and tokenize it
        text = self.texts[idx]
        tokenized_input = tokenizer(text, truncation=True, padding='max_length', max_length=256, return_tensors='pt')

        input_ids = tokenized_input['input_ids'][0]

        # Create labels that are the same shape as input_ids but shifted by one token
        labels = input_ids.clone()
        labels[labels == tokenizer.pad_token_id] = -100  # Mask the padding tokens if any

        return {
            'input_ids': input_ids,
            'labels': labels
        }

## **Splitting Up the Data**

In [8]:
train_dataset = NewsDataset(train_df['text'].tolist(), train_df['is_true'].tolist())
test_dataset = NewsDataset(test_df['text'].tolist(), test_df['is_true'].tolist())

## **Setting up the Trainer**

In this case we will be using HuggingFace's Supervised Fine-tuning Trainer (SFT).

In [9]:
# Defining the training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=16,  # Larger batch size
    gradient_accumulation_steps=4,  # Fewer accumulation steps
    warmup_steps=5, # can change later
    #max_steps=60,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=3407,
    output_dir="outputs",
    report_to='none',
    #num_epochs = 1,
    evaluation_strategy = "epoch",
)


trainer = SFTTrainer( # Can try Trainer instead of SFTTrainer
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,  # Set True if your sequences are short
    args=training_args
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import os
from tqdm import tqdm  # Importing tqdm for the progress bar

# Ensure model is in evaluation mode
model.eval()

# Tokenize and predict using the untrained model
predicted_labels = []
true_labels = []

# Batch processing (use a batch size)
batch_size = 16  # Set an appropriate batch size (adjust if needed)
num_batches = len(test_dataset) // batch_size + (1 if len(test_dataset) % batch_size != 0 else 0)

# Create a progress bar
for i in tqdm(range(num_batches), desc="Evaluating", unit="batch", total=num_batches):
    # Get the batch of samples
    batch_samples = [test_dataset[j] for j in range(i * batch_size, min((i + 1) * batch_size, len(test_dataset)))]

    # Extract input_ids and labels
    batch_input_ids = torch.stack([sample['input_ids'] for sample in batch_samples]).to(model.device)
    batch_true_labels = [sample['labels'].cpu().numpy() for sample in batch_samples]  # True labels for the batch

    # Generate model predictions (even for untrained models)
    with torch.no_grad():
        outputs = model(batch_input_ids)

    # Decode the predictions and map them to labels
    batch_predictions = outputs.logits.argmax(dim=2).cpu().numpy()

    # Process each sample in the batch
    for preds, true_labels_batch in zip(batch_predictions, batch_true_labels):
        # Flatten lists and ignore padding token predictions (-100)
        preds_flat = [pred for pred, label in zip(preds, true_labels_batch) if label != -100]
        true_flat = [label for label in true_labels_batch if label != -100]

        predicted_labels.extend(preds_flat)
        true_labels.extend(true_flat)

# Classification Metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average="weighted")
recall = recall_score(true_labels, predicted_labels, average="weighted")
f1 = f1_score(true_labels, predicted_labels, average="weighted")

# Print Results
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# Confusion Matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Plot Confusion Matrix
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Class 0", "Class 1"], yticklabels=["Class 0", "Class 1"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")

# Save plot to drive
os.makedirs('models', exist_ok=True)
plt.savefig('models/confusion_matrix_untrained.png')
plt.show()

# Save evaluation metrics to a file
eval_results = {
    "accuracy": accuracy,
    "precision": precision,
    "recall": recall,
    "f1_score": f1,
    "confusion_matrix": conf_matrix.tolist()
}

# Save the metrics to a text file
with open('models/evaluation_results_untrained.txt', 'w') as f:
    f.write(f"Accuracy: {accuracy:.2f}\n")
    f.write(f"Precision: {precision:.2f}\n")
    f.write(f"Recall: {recall:.2f}\n")
    f.write(f"F1 Score: {f1:.2f}\n")
    f.write(f"Confusion Matrix:\n{conf_matrix}\n")

print(f"Evaluation results saved to 'models/evaluation_results_untrained.txt' and confusion matrix saved as 'models/confusion_matrix_untrained.png'")


Evaluating: 100%|██████████| 1379/1379 [08:31<00:00,  2.69batch/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.00
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
